# Orbit Propagation Test

In [17]:
# Load tudatpy modules
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.numerical_simulation import environment_setup, propagation_setup
from tudatpy.kernel.astro import element_conversion
from tudatpy.kernel import constants
from tudatpy.util import result2array

In [18]:
# Load other libraries
import numpy as np
import matplotlib.pyplot as plt

In [19]:
# Load spice kernels
spice.load_standard_kernels()

## Environment Setup

In [20]:
# Add Celestial bodies
bodies_to_create = ["Earth", "Moon"]
global_frame_origin = "Moon"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(bodies_to_create, global_frame_origin, global_frame_orientation)

In [21]:
# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

In [22]:
# Add vehicle object to system of bodies
bodies.create_empty_body( "Vehicle" )
# Change parameters of Vehicle
bodies.get_body( "Vehicle" ).mass = 1000.0

## Propagation Setup

In [23]:
# Define propagation
bodies_to_propagate = ["Vehicle"]
central_bodies = ["Moon"]

In [25]:
# Define accelerations acting on Vehicle
acceleration_settings_vehicle = dict(
    Moon=[propagation_setup.acceleration.spherical_harmonic_gravity(4, 0)],
    Earth=[propagation_setup.acceleration.point_mass_gravity()])
acceleration_settings = {"Vehicle": acceleration_settings_vehicle}

In [26]:
# Create acceleration models
acceleration_models = propagation_setup.create_acceleration_models(bodies, acceleration_settings, bodies_to_propagate, central_bodies)

In [34]:
# Set initial conditions for the Vehicle that will be propagated in this simulation.
# The initial conditions are given in Keplerian elements and later on converted to Cartesian elements. Use SI units.
moon_gravitational_parameter = bodies.get("Moon").gravitational_parameter

initial_state = element_conversion.keplerian_to_cartesian_elementwise(
    gravitational_parameter=moon_gravitational_parameter,
    semi_major_axis=2000e3,
    eccentricity=0.5,
    inclination=np.deg2rad(0),
    argument_of_periapsis=np.deg2rad(0),
    longitude_of_ascending_node=np.deg2rad(0),
    true_anomaly=np.deg2rad(0),
)

initial_state

array([1000000.        ,       0.        ,       0.        ,
            -0.        ,    2711.86286209,       0.        ])

In [37]:
# Define time settings
simulation_start_epoch = 0.0
simulation_end_epoch = 5*3600.0

In [38]:
# Integration settings
integrator_settings = propagation_setup.integrator.runge_kutta_4(2.0) # Step size is 2 seconds
termination_settings = propagation_setup.propagator.time_termination(simulation_end_epoch)
dependent_variables_to_save = [propagation_setup.dependent_variable.total_acceleration("Vehicle")]

propagator_settings = propagation_setup.propagator.translational(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    initial_state,
    simulation_start_epoch,
    integrator_settings,
    termination_settings,
    output_variables=dependent_variables_to_save
)

## Propagation Dynamics

In [39]:
# Create simulation object and propagate dynamics.
dynamics_simulator = numerical_simulation.create_dynamics_simulator(bodies, propagator_settings)
results = dynamics_simulator.propagation_results

In [40]:
# Value Results
states = result2array(results.state_history)
states

array([[ 0.00000000e+00,  1.00000000e+06,  0.00000000e+00, ...,
        -0.00000000e+00,  2.71186286e+03,  0.00000000e+00],
       [ 2.00000000e+00,  9.99990184e+05,  5.42370749e+03, ...,
        -9.81608070e+00,  2.71183575e+03,  1.12984826e-03],
       [ 4.00000000e+00,  9.99960736e+05,  1.08473075e+04, ...,
        -1.96316667e+01,  2.71175538e+03,  2.30922583e-03],
       ...,
       [ 1.79960000e+04, -1.84160625e+06,  1.56199879e+06, ...,
        -1.17426120e+03, -4.76554836e+02,  3.00444004e+00],
       [ 1.79980000e+04, -1.84395349e+06,  1.56104459e+06, ...,
        -1.17297880e+03, -4.77641579e+02,  3.00045952e+00],
       [ 1.80000000e+04, -1.84629817e+06,  1.56008822e+06, ...,
        -1.17169664e+03, -4.78726077e+02,  2.99648039e+00]])

In [41]:
# Define the radius of the sphere
r = 1737.4e3

# Define the angles for the phi and theta parameters
phi = np.linspace(0, 2 * np.pi, 20)
theta = np.linspace(0, np.pi, 20)

# Create a meshgrid of phi and theta values
phi, theta = np.meshgrid(phi, theta)

# Calculate the x, y, and z coordinates for each point on the sphere
xM = r * np.sin(theta) * np.cos(phi)
yM = r * np.sin(theta) * np.sin(phi)
zM = r * np.cos(theta)

In [42]:
%matplotlib qt

# Define a 3D figure using pyplot
fig = plt.figure(figsize=(6,6), dpi=125)
ax = fig.add_subplot(111, projection='3d')

# Plot the positional state history
ax.plot(states[:, 1], states[:, 2], states[:, 3], label=bodies_to_propagate[0], linestyle='-.')
ax.plot_surface(xM, yM, zM, color='gray', alpha=0.5)
ax.scatter(0.0, 0.0, 0.0, label="Moon", marker='o', color='gray')

# Add the legend and labels, then show the plot
ax.legend()
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.axis('equal')
plt.show()